In [1]:
from pynq import Overlay
from pynq.lib.video import *
import time
import PIL.Image

overlay = Overlay("/home/xilinx/base14A.bit")

In [2]:
imgen_vdma = overlay.video.axi_vdma_0.readchannel

In [3]:
videoMode = common.VideoMode(1280, 720, 24) # (640, 480, 24) (1280, 720, 24)
imgen_vdma.mode = videoMode
imgen_vdma.start()

In [4]:
#NEW CELL I (AJAY) ADDED NOT ORGINAL COPY PASTEOF COMMENTED OUT CODE!
hdmi_out = overlay.video.hdmi_out
hdmi_out._vdma = overlay.video.axi_vdma #Use the correct VDMA!
hdmi_out.configure(videoMode)
hdmi_out.start()

In [5]:
pixgen = overlay.pixel_generator_0

In [ ]:
import socket
import pickle

# Initialize TCP client
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('127.0.0.1', 9999))
print("Connected to server.")

pixgen.register_map.PAUSE = 0x0000 # PAUSE FLAG
counter=0
while True:
     if (counter<720):
          pixgen.register_map.PAUSE = 0x0001
          serial_row = client_socket.recv(1431)
          row=pickle.loads(serial_row)
          counterd = 0
          for data in row:
               if data:
                    row[counterd] = 1
               else:
                    row[counterd] = 0
               counterd = counterd + 1
          binary_str = ''.join(map(str, row))
          while len(binary_str) % 8 != 0:
               binary_str = '0' + binary_str
          pixgen.register_map.ADDR=hex(counter)
          for i in range(40):
               hex_int = int(binary_str[32*i:((i+1)*32)-1], 2)
               register_name = f"gp{i}"
               setattr(pixgen.register_map, register_name, hex_int)
          counter=counter+1
     string=client_socket.recv(256)
     message = pickle.loads(string)
     if message=="P":
          pass
     else:
          hex_str = str(pixgen.register_map.PAUSE)
          if(int(hex_str, 16) == 0):
               pixgen.register_map.PAUSE = 0x0000
          if(int(hex_str, 16) == 1):
               pixgen.register_map.PAUSE = 0x0000
     frame = imgen_vdma.readframe() # Output next frame
     hdmi_out.writeframe(frame)